In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!wget -q https://archive.apache.org/dist/spark/spark-2.2.3/spark-2.2.3-bin-hadoop2.7.tgz
!tar xf spark-2.2.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
!ls

gdrive	sample_data  spark-2.2.3-bin-hadoop2.7	spark-2.2.3-bin-hadoop2.7.tgz


In [4]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)


In [5]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [6]:
!java -version

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.2.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

In [0]:
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("MLlib")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
spark = SparkSession(sc)

Q1

In [0]:
payment_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/hw9/payment.csv')
payment_df = payment_df.alias('payment_df')


In [11]:
payment_df.groupby('customer_id').agg({"amount":"sum"}).orderBy('sum(amount)',ascending = False).show(3)

+-----------+------------------+
|customer_id|       sum(amount)|
+-----------+------------------+
|        526| 221.5500000000001|
|        148| 216.5400000000001|
|        144|195.58000000000007|
+-----------+------------------+
only showing top 3 rows



Q2

In [0]:
rental_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/hw9/rental.csv')
rental_df = rental_df.alias('rental_df')


In [13]:
rental_df.orderBy('return_date',ascending = True).show(3)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|       32|2005-05-25 04:06:21|        3832|        230|2005-05-25 23:55:21|       1|2006-02-15 21:30:53|
|       21|2005-05-25 01:59:46|         146|        388|2005-05-26 01:01:46|       2|2006-02-15 21:30:53|
|       14|2005-05-25 00:31:15|        2701|        446|2005-05-26 02:56:15|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
only showing top 3 rows



Q3

In [14]:
rental_df.groupBy('staff_id').count().show()

+--------+-----+
|staff_id|count|
+--------+-----+
|       1| 8040|
|       2| 8004|
+--------+-----+



Q4

In [0]:
staff_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/hw9/staff.csv')
staff_df = staff_df.alias('staff_df')


In [16]:
df1 = rental_df.groupBy('staff_id').count()
df2 = staff_df.select('staff_id','first_name','last_name')
df2.join(df1, df2.staff_id == df1.staff_id,"inner").select('first_name','last_name','count').show()


+----------+---------+-----+
|first_name|last_name|count|
+----------+---------+-----+
|      Mike|  Hillyer| 8040|
|       Jon| Stephens| 8004|
+----------+---------+-----+



Q5

In [0]:
inventory_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/hw9/inventory.csv')
inventory_df = inventory_df.alias('inventory_df')


In [0]:
store_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/hw9/store.csv')
store_df = store_df.alias('store_df')


In [0]:
address_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/hw9/address.csv')
address_df = address_df.alias('address_df')


In [0]:
customer_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/hw9/customer.csv')
customer_df = customer_df.alias('customer_df')


In [0]:
film_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/hw9/film.csv')
film_df = film_df.alias('film_df')


In [0]:
stepone = rental_df.join(inventory_df,rental_df.inventory_id == inventory_df.inventory_id,"inner")


In [0]:
steptwo = stepone.join(store_df,stepone.store_id == store_df.store_id,"inner")


In [0]:
stepthree = steptwo.join(address_df,steptwo.address_id == address_df.address_id,"inner")


In [0]:
stepfour = stepthree.join(customer_df,stepthree.customer_id == customer_df.customer_id,"inner").select('rental_date','return_date','address','staff_id','film_id','first_name','last_name')


In [0]:
stepfive = stepfour.join(staff_df,stepfour.staff_id == staff_df.staff_id,"inner").drop('staff_id','address_id','picture','email','store_id','active','username','password','last_update')


In [27]:
stepsix = stepfive.join(film_df,stepfive.film_id == film_df.film_id,"inner").drop('film_id','release_year','language_id','original_language_id','rental_duration','rental_rate','length','replacement_cost','rating','special_features','last_update')
stepsix.show(3)

+-------------------+-------------------+------------------+----------+---------+----------+---------+---------------+--------------------+
|        rental_date|        return_date|           address|first_name|last_name|first_name|last_name|          title|         description|
+-------------------+-------------------+------------------+----------+---------+----------+---------+---------------+--------------------+
|2005-05-24 22:53:30|2005-05-26 22:04:30| 47 MySakila Drive| CHARLOTTE|   HUNTER|      Mike|  Hillyer|BLANKET BEVERLY|A Emotional Docum...|
|2005-05-24 22:54:33|2005-05-28 19:40:33|28 MySQL Boulevard|     TOMMY|  COLLAZO|      Mike|  Hillyer|   FREAKY POCUS|A Fast-Paced Docu...|
|2005-05-24 23:03:39|2005-06-01 22:12:39|28 MySQL Boulevard|    MANUEL|  MURRELL|      Mike|  Hillyer|  GRADUATE LORD|A Lacklusture Epi...|
+-------------------+-------------------+------------------+----------+---------+----------+---------+---------------+--------------------+
only showing top 3 r

In [28]:
stepsix.orderBy('return_date',ascending = True).show(3)

+-------------------+-------------------+------------------+----------+---------+----------+---------+------------------+--------------------+
|        rental_date|        return_date|           address|first_name|last_name|first_name|last_name|             title|         description|
+-------------------+-------------------+------------------+----------+---------+----------+---------+------------------+--------------------+
|2005-05-25 04:06:21|2005-05-25 23:55:21|28 MySQL Boulevard|       JOY|   GEORGE|      Mike|  Hillyer| STALLION SUNDANCE|A Fast-Paced Tale...|
|2005-05-25 01:59:46|2005-05-26 01:01:46| 47 MySakila Drive|     CRAIG|  MORRELL|       Jon| Stephens|     APACHE DIVINE|A Awe-Inspiring R...|
|2005-05-25 00:31:15|2005-05-26 02:56:15|28 MySQL Boulevard|  THEODORE|     CULP|      Mike|  Hillyer|MONTEREY LABYRINTH|A Awe-Inspiring D...|
+-------------------+-------------------+------------------+----------+---------+----------+---------+------------------+--------------------+

Q6

In [29]:
payment_df.groupby('customer_id').agg({"amount":"sum"}).where("sum(amount)>='200'").show()

+-----------+-----------------+
|customer_id|      sum(amount)|
+-----------+-----------------+
|        526|221.5500000000001|
|        148|216.5400000000001|
+-----------+-----------------+



Q7

In [30]:
df4 = payment_df.groupby('customer_id').agg({"amount":"sum"}).where("sum(amount)>='200'")
df4.join(customer_df, df4.customer_id == customer_df.customer_id,"inner").select('first_name','last_name','sum(amount)').show()

+----------+---------+-----------------+
|first_name|last_name|      sum(amount)|
+----------+---------+-----------------+
|      KARL|     SEAL|221.5500000000001|
|   ELEANOR|     HUNT|216.5400000000001|
+----------+---------+-----------------+



Q8

In [0]:
import pyspark.sql.functions as func

In [0]:
df5 = rental_df.withColumn('rental_years',func.year(func.to_timestamp('rental_date'))).select('rental_years','customer_id').where("rental_years=='2005'")
df6 = rental_df.withColumn('rental_years',func.year(func.to_timestamp('rental_date'))).select('rental_years','customer_id').where("rental_years=='2006'")

In [49]:
df5.select('customer_id').subtract(df6.select('customer_id')).count()

441

In [50]:
df6.select('customer_id').subtract(df5.select('customer_id')).count()

0